In [14]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split

data_path = Path("./winequality-white.csv")
data_columns = ["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density", 
"pH", "sulphates", "alcohol", "quality"]
wine_df = pd.read_csv(data_path, header=0, names=data_columns, sep=";")

test_size=0.2
train_df, test_df = train_test_split(wine_df, test_size=test_size)

# data overview:
#print(wine_df.describe())

# which quality classes do we have? :
qualities = wine_df["quality"].unique()
print(f"Number of unique 'qualities': {len(qualities)}")
print(f"Qualities: {sorted(qualities)}")
binc = np.bincount([q for q in wine_df["quality"]])
no_inst = len(wine_df)
print(f"\nClass counts: {binc}")
print(f"\nNumber of instances: {no_inst} ")
print(f"\nClass fractions: {np.round(binc/no_inst,4) * 100}")


Number of unique 'qualities': 7
Qualities: [3, 4, 5, 6, 7, 8, 9]

Class counts: [   0    0    0   20  163 1457 2198  880  175    5]

Number of instances: 4898 

Class fractions: [ 0.    0.    0.    0.41  3.33 29.75 44.88 17.97  3.57  0.1 ]


#### !Refactoring: don't use pandas dataframes, where not necessary! ####

In [15]:
from sklearn.preprocessing import StandardScaler

def scale_dataframe(data_df, exempt_last_column=False, column_names_to_scale=None):
    """
        Scales columns of a given data frame with a StandardScaler from Sklearn. 
        Input:
            data_df : dataframe with numerical values to normalize
            exempt_last_column : if true, column_names_to_scale will be ignored and all but the last column will be scaled.
            column_names_to_scale : list of the names of the columns to be scaled

        Output:
            dataframe with columns scaled
    """
    scaler = StandardScaler()

    if(exempt_last_column & (column_names_to_scale != None)):
        raise UserWarning("exempt_last_column=True : your column_names_to_scale will be ignored!")
    if(exempt_last_column):
        data = data_df.to_numpy()
        data_to_scale = data[:,:-1]
        last_column = np.expand_dims(data[:,-1].astype(np.int_), axis=1)
        data_scaled = np.append(scaler.fit_transform(data_to_scale), last_column, axis=1)
        return pd.DataFrame(data_scaled)
    elif(column_names_to_scale):
        data_to_scale = data_df[column_names_to_scale].to_numpy()
        data_scaled = scaler.fit_transform(data_to_scale)
        df_temp = pd.DataFrame(data_scaled, columns=column_names_to_scale, index=data_df.index)
        data_df[column_names_to_scale]= df_temp
    else:
        data_to_scale = data_df.to_numpy()
        data_scaled = scaler.fit_transform(data_to_scale)
        data_df = pd.DataFrame(data_scaled)
    
    return data_df


In [16]:
"""
# Model definition from the WineDataset note book -- I don't know how to import this from another Jupyter notebook...
from torch import nn

class WineNetwork(nn.Module):
    def __init__(self):
        super(WineNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(11, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(p=0.2),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Linear(256, 10),
            nn.ReLU()
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits
"""

"\n# Model definition from the WineDataset note book -- I don't know how to import this from another Jupyter notebook...\nfrom torch import nn\n\nclass WineNetwork(nn.Module):\n    def __init__(self):\n        super(WineNetwork, self).__init__()\n        self.linear_relu_stack = nn.Sequential(\n            nn.Linear(11, 64),\n            nn.ReLU(),\n            nn.BatchNorm1d(64),\n            nn.Dropout(p=0.2),\n            nn.Linear(64, 128),\n            nn.ReLU(),\n            nn.BatchNorm1d(128),\n            nn.Linear(128, 256),\n            nn.ReLU(),\n            nn.BatchNorm1d(256),\n            nn.Linear(256, 10),\n            nn.ReLU()\n        )\n\n    def forward(self, x):\n        logits = self.linear_relu_stack(x)\n        return logits\n"

In [17]:
def train_loop(dataloader, model, loss_fn, optimizer):
    losses, nof_correct = 0, 0
    for xx, y_true in dataloader:
        y_pred = model(xx)
        loss= loss_fn(y_pred, y_true)
        losses += loss.item()
        nof_correct += (y_pred.argmax(1) == y_true).sum().item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return losses, nof_correct

def test_loop(dataloader, model, loss_fn):
    losses, no_correct = 0, 0
    with torch.no_grad():
        for (X,y_true) in dataloader:
            pred = model(X)
            losses += loss_fn(pred, y_true).item()
            no_correct += (pred.argmax(1)== y_true).sum().item()
     
    return losses, no_correct
    

In [18]:
# transform the training data with the decision-tree:

import pickle


def Wine_tree_map(data_df, tree_name, does_predict_proba=False):
    """
    Maps a data-frame by using a pre-trained decision-tree.
    Input:
        data_df : Pandas data-frame to be mapped
        tree_name : path / name of the pretrained tree
        does_predict_proba : uses the predict_proba method if true, uses predict otherwize
    Returns:
        Pandas data-frame
    """
    tree_model = pickle.load(open(tree_name, 'rb'))
    X = data_df.to_numpy()
    if(does_predict_proba):
        # "np.int_" is "long" in numpy:
        Z = np.append(tree_model.predict_proba(X[:,:-1]), np.expand_dims(X[:,-1].astype(np.int_), axis=1), axis=1)
    else:
        Z = np.append(np.expand_dims(tree_model.predict(X[:,:-1]).astype(np.int_), axis=1), np.expand_dims(X[:,-1].astype(np.int_), axis=1), axis=1)
    
    return pd.DataFrame(Z)

In [19]:
# maps the training data with the neural-net:

import numpy as np
import pandas as pd


def WineNetwork_map(data_df, net_name):
    """
    Maps a dataframe by applying a pre-trained neural net of type "class WineNetwork"
    Input:
        data_df : the pandas data-frame to be transformed
        net_name : the path / name of the net-model of type WineNetwork to be loaded
    Returns:
        a pandas data-frame        
    """
    class WineNetwork(nn.Module):
        def __init__(self):
            super(WineNetwork, self).__init__()
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(11, 64),
                nn.ReLU(),
                nn.BatchNorm1d(64),
                nn.Dropout(p=0.2),
                nn.Linear(64, 128),
                nn.ReLU(),
                nn.BatchNorm1d(128),
                nn.Linear(128, 256),
                nn.ReLU(),
                nn.BatchNorm1d(256),
                nn.Linear(256, 10),
                nn.ReLU()
            )

        def forward(self, x):
            logits = self.linear_relu_stack(x)
            return logits

    net_model = WineNetwork()
    net_model.load_state_dict(torch.load(net_name))
    net_model.eval()

    # add softmax for prediction of probabilities:
    # softm = lambda x : np.exp(x)/np.sum(np.exp(x))
    softm = torch.nn.Softmax(dim=1)
    
    X = torch.tensor(data_df.iloc[:,:-1].to_numpy(), dtype=torch.float32).detach()
    Y = torch.tensor(data_df.iloc[:, -1].to_numpy(), dtype=torch.long).detach()

    Z = torch.cat((softm(net_model(X)), Y.unsqueeze(dim=1)), dim=1).detach().numpy()
    
    return pd.DataFrame(Z)
    

In [20]:
# We use the boiler-plate triade: pandas.data-frame -> torch.data-set -> torch.data-loader

# define torch.dataset: __init__(), __len__(), __getitem__()
from torch.utils.data import Dataset

class WineDataSet(Dataset):
    def __init__(self, data_df, transform=None, target_transform=None):
        self.data_df = data_df
        self.transform = transform
        self.target_transform = target_transform
        self.X = torch.tensor(self.data_df.iloc[:,:-1].to_numpy(), dtype=torch.float32)
        self.Y = torch.tensor(self.data_df.iloc[:, -1].to_numpy(), dtype= torch.long)

    def __len__(self):
        return len(self.Y)
        
    def __getitem__(self,idx):
        self.x = self.X[idx,:]
        self.y = self.Y[idx]
        if self.transform != None:
            self.x = self.transform(self.x)
        if self.target_transform != None:
            self.y = self.target_transform(self.y)
        return self.x, self.y

In [21]:
"""
# LOAD AND PREPARE DATA: DATA FRESH LOAD - NOT for stacking:

from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from torch import nn
from torch.utils.data import DataLoader


# DATA FRESH LOAD - NOT for stacking:
data_path = Path("./winequality-white.csv")
column_names = ["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density", 
"pH", "sulphates", "alcohol", "quality"]
#column_names_to_normalize = column_names[:-1]
wine_df = pd.read_csv(data_path, header=0, names=column_names, sep=";")

# prepare data:
shuffle(wine_df, random_state=0)
scaled_wine_df = scale_dataframe(wine_df, exempt_last_column=True)

# names/ paths of the pre-trained classifiers:
#WineNetwork_filename = "model_640_0.001_369_64_SGD.pt"
NET_FILENAME="model_635_0.001_255_64_SGD.pt"
TREE_FILENAME= 'AdaBoost_071_model.dct'

# use the scaled data for the net:
net_df = WineNetwork_map(scaled_wine_df, net_name=NET_FILENAME)
net_df = scale_dataframe(net_df)

# use the unscaled data for the tree:
tree_df = Wine_tree_map(wine_df, tree_name=TREE_FILENAME, does_predict_proba=True)
tree_df = scale_dataframe(tree_df)

# combine net_df and tree_df to combined_data_df:
combined_data_df = pd.DataFrame(np.append(net_df.iloc[:,:-1].to_numpy(), tree_df.to_numpy(), axis=1))

# split into train_df, test_df:
TEST_SIZE = 0.2
combined_train_df, combined_test_df = train_test_split(combined_data_df, test_size=TEST_SIZE)

# calculate the input-dimension for the blender - minus one is for the label column:
blender_input_dim = combined_data_df.shape[1]-1

# create dataloader from train_df and test_df:
BATCH_SIZE=64
train_ds = WineDataSet(combined_train_df)
test_ds = WineDataSet(combined_test_df)
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=True)
"""

'\n# LOAD AND PREPARE DATA: DATA FRESH LOAD - NOT for stacking:\n\nfrom pathlib import Path\nfrom sklearn.model_selection import train_test_split\nfrom sklearn.utils import shuffle\nfrom torch import nn\nfrom torch.utils.data import DataLoader\n\n\n# DATA FRESH LOAD - NOT for stacking:\ndata_path = Path("./winequality-white.csv")\ncolumn_names = ["fixed acidity", "volatile acidity", "citric acid", "residual sugar", "chlorides", "free sulfur dioxide", "total sulfur dioxide", "density", \n"pH", "sulphates", "alcohol", "quality"]\n#column_names_to_normalize = column_names[:-1]\nwine_df = pd.read_csv(data_path, header=0, names=column_names, sep=";")\n\n# prepare data:\nshuffle(wine_df, random_state=0)\nscaled_wine_df = scale_dataframe(wine_df, exempt_last_column=True)\n\n# names/ paths of the pre-trained classifiers:\n#WineNetwork_filename = "model_640_0.001_369_64_SGD.pt"\nNET_FILENAME="model_635_0.001_255_64_SGD.pt"\nTREE_FILENAME= \'AdaBoost_071_model.dct\'\n\n# use the scaled data for 

In [22]:
# LOAD AND PREPARE DATA FOR STACKING AND BLENDER TRAINING:

#from pathlib import Path
#from sklearn.model_selection import train_test_split
#from sklearn.utils import shuffle
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader



# DATA LOAD FOR STACKING:
train_df = pd.read_csv("./train.csv")
test_df = pd.read_csv("./test.csv")

# names/ paths of the pre-trained classifiers:
NET_FILENAME = "net_model_647_0.001_393_64_SGD.pt"
TREE_FILENAME = 'AdaBoost_071_model.dct'
TREE_FILENAME2 = "DecisionTree_061_model.dct"


# TRAIN DATA PREPARATION:
# use the scaled data for the net:
scaled_train_df = scale_dataframe(train_df, exempt_last_column=True)
net_train_df = WineNetwork_map(scaled_train_df, net_name=NET_FILENAME)
net_train_df = scale_dataframe(net_train_df, exempt_last_column=True)

# use the unscaled data for the tree:
tree_train_df = Wine_tree_map(train_df, tree_name=TREE_FILENAME, does_predict_proba=True)
zer = torch
tree_train_df = scale_dataframe(tree_train_df, exempt_last_column=True)

tree2_train_df = Wine_tree_map(train_df, tree_name=TREE_FILENAME2, does_predict_proba=True)
tree2_train_df = scale_dataframe(tree_train_df, exempt_last_column=True)


# combine net_df and tree_df to combined_data_df:
combined_train_ = np.append(net_train_df.iloc[:,:-1].to_numpy(), tree_train_df.iloc[:,:-1].to_numpy(), axis=1)
combined_train_df = pd.DataFrame(np.append(combined_train_, tree2_train_df.to_numpy(), axis=1))

# TEST DATA PREPARATION:
# use the scaled data for the net:
scaled_test_df = scale_dataframe(test_df, exempt_last_column=True)
net_test_df = WineNetwork_map(scaled_test_df, net_name=NET_FILENAME)
net_test_df = scale_dataframe(net_test_df, exempt_last_column=True)

# use the unscaled data for the tree:
tree_test_df = Wine_tree_map(test_df, tree_name=TREE_FILENAME, does_predict_proba=True)
tree_test_df = scale_dataframe(tree_test_df, exempt_last_column=True)

tree2_test_df = Wine_tree_map(test_df, tree_name=TREE_FILENAME2, does_predict_proba=True)
tree2_test_df = scale_dataframe(tree_test_df, exempt_last_column=True)

# combine net_df and tree_df to combined_data_df:
combined_test_ = np.append(net_test_df.iloc[:,:-1].to_numpy(), tree_test_df.iloc[:,:-1].to_numpy(), axis=1)
combined_test_df = pd.DataFrame(np.append(combined_test_, tree2_test_df.to_numpy(), axis=1))


# PARAMETERS:
# calculate the input-dimension for the blender - minus one is for the label column:
BLENDER_INPUT_DIM = combined_train_df.shape[1]-1

# create dataloader from train_df and test_df:
BATCH_SIZE=256
train_ds = WineDataSet(combined_train_df)
test_ds = WineDataSet(combined_test_df)
train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=BATCH_SIZE, shuffle=True)

In [26]:
import torch
from torch import nn

class BlenderModel(nn.Module):
    
    def __init__(self, input_dim):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
        nn.Linear(input_dim, 128),
        nn.ReLU(),
        nn.BatchNorm1d(128),
        nn.Dropout(p=0.2),
        nn.Linear(128, 256),
        nn.ReLU(),
        nn.BatchNorm1d(256),
        nn.Dropout(p=0.2),
        nn.Linear(256, 10),
        nn.ReLU(),
        )
    
    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits


In [48]:
# Train the blender model:
import os

from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter


# writer for tensorboard:
writer = SummaryWriter()


# blender model and parameters:

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

blender_model = BlenderModel(BLENDER_INPUT_DIM)

# parameters:

# loss function:
# cross-entropy:
loss_fn = nn.CrossEntropyLoss()

# optimizer:

# adam:
#OPTIMIZER_NAME = "ADAM"
#LEARNING_RATE = 1e-4
#optimizer = torch.optim.Adam(blender_model.parameters(), lr=LEARNING_RATE, weight_decay=0.5, amsgrad=False)
#optimizer = torch.optim.Adam(blender_model.parameters(), lr=LEARNING_RATE)

# sgd:
#OPTIMIZER_NAME = "SGD"
#LEARNING_RATE = 1e-3
#MOMENTUM=0.9
#optimizer = torch.optim.SGD(blender_model.parameters(), lr= LEARNING_RATE, momentum=MOMENTUM)

# adamax:
#OPTIMIZER_NAME = "ADAMAX"
#LEARNING_RATE = 1e-4
#optimizer = torch.optim.Adamax(blender_model.parameters(), lr=LEARNING_RATE, weight_decay=0.8)

# LBFGS
#OPTIMIZER_NAME = "LBFGFS"
#LEARNING_RATE = 1
#MAX_ITER = 20
#optimizer = torch.optim.LBFGS(params, lr=LEARNING_RATE, max_iter=MAX_ITER, max_eval=None, tolerance_grad=1e-07, tolerance_change=1e-09, history_size=100, line_search_fn=None)


EPOCHS = 120
WRITE_LOG_AFTER_EPOCHS = 30
best_model_name = ""
max_correct = -torch.inf


for ep in range(1, EPOCHS+1):        
       
        # put model in train mode:
        blender_model.train()
        (train_loss, train_no_correct) = train_loop(train_dl, blender_model, loss_fn, optimizer)
              
        # switch model to to evaluation mode:
        blender_model.eval()
        (test_loss, test_no_correct) = test_loop(test_dl, blender_model, loss_fn)

        if(test_no_correct > max_correct):
            max_correct = test_no_correct
            if(best_model_name):
                os.remove(best_model_name)
            best_model_name = "./blender_model_" + str(test_no_correct) + "_" + str(LEARNING_RATE) + "_" + str(ep) + "_" + str(BATCH_SIZE) + "_" + OPTIMIZER_NAME + "_sv.pt"
            torch.save(blender_model.state_dict(), best_model_name)

        writer.add_scalar("Loss/test", test_loss/ len(test_ds), ep)
        writer.add_scalar("Accuracy/test", test_no_correct/ len(test_ds), ep)
        writer.add_scalar("Loss/train", train_loss/ len(train_ds), global_step=ep)
        writer.add_scalar("Accuracy/train", train_no_correct/ len(train_ds), global_step=ep)
       
        if ep % WRITE_LOG_AFTER_EPOCHS == 0:
            print(f"\n----- Epoch: {ep} -----")
            print(f"Epoch loss: {test_loss/ len(test_ds)}")
            print(f"Epoch accuracy: {test_no_correct/ len(test_ds)}")
            

Using device: cpu

----- Epoch: 30 -----
Epoch loss: 0.004966189788312328
Epoch accuracy: 0.7091836734693877

----- Epoch: 60 -----
Epoch loss: 0.005227208137512207
Epoch accuracy: 0.7081632653061225

----- Epoch: 90 -----
Epoch loss: 0.00523871365858584
Epoch accuracy: 0.7071428571428572

----- Epoch: 120 -----
Epoch loss: 0.005277684270119181
Epoch accuracy: 0.7071428571428572


#### Some test statistics: ####
Names of Models are constructed according to: <br>
best_model_name = "./blender_model_" + str(test_no_correct) + "_" + str(LEARNING_RATE) + "_" + str(ep) + "_" + str(BATCH_SIZE) + "_" + OPTIMIZER_NAME + "_sv.pt <br>
<br>
blender_model_693_0.0001_48_256_SGD_sv.pt <br>
blender_model_696_0.0001_15_512_ADAM_sv.pt <br>
blender_model_696_1e-05_219_1024_ADAM_sv.pt <br>
blender_model_697_0.0001_16_512_ADAM_sv.pt <br>
blender_model_697_0.0001_44_256_SGD_sv.pt <br>
blender_model_698_0.0001_11_256_ADAM_sv.pt <br>
blender_model_698_1e-05_100_512_ADAM_sv.pt <br>
blender_model_701_0.0001_36_256_SGD_sv.pt <br>
blender_model_701_1e-05_192_512_ADAM_sv.pt <br>
blender_model_697_0.001_13_256_ADAMAX_sv.pt <br>
<br>
The blender was feed by the following three models: <br>
<br>
AdaBoost_071_model.dct <br>
DecisionTree_061_model.dct <br>
net_model_647_0.001_393_64_SGD.pt <br>
<br>
with 71%, 61% and 64% correct prediction rate (corresponding to 647 correct predictions). <br>
<br>
Summary: I would have expected the blender to be considerably better than the best of the ingoing predictors, especially since the net and the decision-tree 
are so much different models (i.e. different by their construction, but maybe not by their prediction behaviour... needs to be investigated...)




